# Triple_check

In [1]:
from collections import defaultdict
from operator import itemgetter
import csv

## Read train data and make dicts for check

In [2]:
f = open('dataset/train.csv', 'r')
f.readline()

total = 0

first_check = defaultdict(lambda: defaultdict(int))
second_check = defaultdict(lambda: defaultdict(int))
last_check = defaultdict(lambda: defaultdict(int))

flag = 0

while 1:
    line = f.readline().strip()
    total += 1
     
    if line == '':
        print 'total lines: {}'.format(total)
        flag = 1
        break
    
    content = line.split(',')

    user_location_country = content[3]
    srch_destination_id = content[16]
    is_booking = int(content[18])
    cnt = int(content[19])
    hotel_country = content[21]
    hotel_market = content[22]
    hotel_cluster = content[23]
    
    score = is_booking * 10 + cnt
    
    f1 = (user_location_country, hotel_country)
    s2 = (hotel_country, srch_destination_id)
    l3 = (srch_destination_id, hotel_market)
    
    if hotel_cluster in first_check[f1].keys():
        first_check[f1][hotel_cluster] += score
    else :
        first_check[f1][hotel_cluster] = score
        
    if hotel_cluster in second_check[s2].keys():
        second_check[s2][hotel_cluster] += score
    else :
        second_check[s2][hotel_cluster] = score

    if hotel_cluster in last_check[l3].keys():
        last_check[l3][hotel_cluster] += score
    else :
        last_check[l3][hotel_cluster] = score
    
f.close()


total lines: 37670294


### if flag is changed to '1', start to make a submission

In [3]:
flag

1

In [ ]:
if flag == 1:
    f_t = open('dataset/test.csv', 'r')
    f_t.readline()
    total_t = 0
    
    while 1:
        line = f_t.readline().strip()
        total_t += 1
        
        if total_t % 100000 == 0:
            %time
            print 'Reading...'
            
        if line == '':
            print 'test total lines: {}'.format(total_t)
            %%time
            print 'complete'
            break

        content = line.split(',')
        
        id_t = content[0]
        user_location_country = content[4]
        srch_destination_id = content[17]
        hotel_country = content[20]
        hotel_market = content[21]

        f1 = (user_location_country, hotel_country)
        s2 = (hotel_country, srch_destination_id)
        l3 = (srch_destination_id, hotel_market)
        
        if f1 in first_check.keys():
            first_report = sorted(first_check[f1].items(), key=itemgetter(1), reverse=True)[:5]
        
        if s2 in second_check.keys():
            second_report = sorted(second_check[s2].items(), key=itemgetter(1), reverse=True)[:5]
            
        if l3 in last_check.keys():
            last_report = sorted(last_check[l3].items(), key=itemgetter(1), reverse=True)[:5]
            
        report_set = [first_report, second_report, last_report]
        
        final_report_temp = defaultdict(int) 
        
        for report in report_set:
            for item in report:
                if item[0] in final_report_temp.keys():
                    final_report_temp[item[0]] += item[1]
                else:
                    final_report_temp[item[0]] = item[1]
        
        final_report_1 = sorted(final_report_temp.items(), key=itemgetter(1), reverse=True)[:5]
        final_report = []
        
        for item in final_report_1:
            final_report.append(item[0])
        
        with open('test_submission.csv', 'a') as f:
            writer = csv.writer(f)
            writer.writerow((id_t, final_report))

        
f.close()


Wall time: 0 ns
Reading...
Wall time: 0 ns
Reading...
Wall time: 0 ns
Reading...
Wall time: 0 ns
Reading...


In [8]:
import dask.dataframe as dd

In [9]:
test_report = dd.read_csv('dataset/train.csv')


In [10]:
test_report.get_partition(0).head()

,date_time,site_name,posa_continent,user_location_country,user_location_region,user_location_city,orig_destination_distance,user_id,is_mobile,is_package,...,srch_children_cnt,srch_rm_cnt,srch_destination_id,srch_destination_type_id,is_booking,cnt,hotel_continent,hotel_country,hotel_market,hotel_cluster
0,2014-08-11 07:46:59,2,3,66,348,48862,2234.2641,12,0,1,...,0,1,8250,1,0,3,2,50,628,1
1,2014-08-11 08:22:12,2,3,66,348,48862,2234.2641,12,0,1,...,0,1,8250,1,1,1,2,50,628,1
2,2014-08-11 08:24:33,2,3,66,348,48862,2234.2641,12,0,0,...,0,1,8250,1,0,1,2,50,628,1
3,2014-08-09 18:05:16,2,3,66,442,35390,913.1932,93,0,0,...,0,1,14984,1,0,1,2,50,1457,80
4,2014-08-09 18:08:18,2,3,66,442,35390,913.6259,93,0,0,...,0,1,14984,1,0,1,2,50,1457,21


In [ ]:
length = 0
for num in range(test_report.npartitions):
    test_sample = test_report.get_partition(num)
    length += len(test_sample)
    print length

result length(accumulate)
309560
618036
926315
1232672
1538200
1843710
2148756
2454614
**final: 2528243**

submission의 len이 총 2,528,243 나와야 함. 

In [2]:
import dask.dataframe as dd

test = dd.read_csv('test_submission.csv')

In [5]:
len(test)

1143483